In [1]:
#!pip install fastapi uvicorn

In [2]:
import uvicorn
from fastapi import FastAPI
import numpy as np
import pandas as pd
import pickle
from pydantic import BaseModel

In [3]:
description = """
Mall segmentation API helps you to determine which group the customer belongs to.

# Items

You can find which category the customer is in

# Users

Just enter the fields below in the predict tab to make predictions

All values are to be input as integers

**Annual income**: Annual income of a customer in 1000s

**Spending score**: Spending Score is assigned to the customer based on parameters like customer behavior and purchasing data. Enter values from 0 - 100(100 included)

"""

In [4]:
# Initializing the app object
app = FastAPI(title= "Mall customer group",
              description= description)

In [5]:
# Opening and loading pickled files
pickle_in = open('model_kmeans.pkl',"rb")
model = pickle.load(pickle_in)
scalar=pickle.load(open('scaling_kmeans.pkl','rb'))

In [6]:
@app.get('/')
def index():
    return {'Message': 'Welcome to Customer group prediction. Type /docs at the end of the above url'}

In [7]:
@app.get('/{name}')
def get_name(name: str):
    return {'Hello, Please input your name here': f'{name}'}

In [8]:
class group(BaseModel):
    annual_income: int
    spending_score: int

In [9]:
@app.post('/predict')
def predict_group(data: group):
    data = data.dict()
    annual_income = data['annual_income']
    spending_score = data['spending_score']
    
    prediction = model.predict([[annual_income, spending_score]])
    print(np.array(list(data.values())).reshape(1,-1))
    scaled_data=scalar.transform(np.array(list(data.values())).reshape(1,-1))
    prediction = model.predict(scaled_data)
    if (prediction[0] == 0):
        prediction = "Customer with medium annual income and medium spending score"
    elif (prediction[0] == 1):
        prediction = "Customer with high annual income and low spending score"
    elif (prediction[0] == 2):
        prediction = "Customer with low annual income and low spending score"
    elif (prediction[0] == 3):
        prediction = "Customer with low annual income and high spending score"
    elif (prediction[0] == 4):
        prediction = "Customer with high annual income and high spending score"
        
        
    return {
        'prediction': prediction
    }
    

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
if __name__ == '__main__':
    uvicorn.run(app, host = '127.0.0.1', port = 8000)

INFO:     Started server process [12748]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54335 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:54335 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     127.0.0.1:54336 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54336 - "GET /openapi.json HTTP/1.1" 200 OK
[[20 65]]
INFO:     127.0.0.1:54348 - "POST /predict HTTP/1.1" 200 OK
[[90 78]]
INFO:     127.0.0.1:54355 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:54725 - "GET /favicon.ico HTTP/1.1" 200 OK
[[20 65]]
INFO:     127.0.0.1:54765 - "POST /predict HTTP/1.1" 200 OK
[[90 78]]
INFO:     127.0.0.1:54795 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [12748]
